In [1]:
import holmes.experiment as Ex
import tempfile
import pprint
import json
import os
from terrarium.flatten import flatten_dict, expand_dict
import tempfile

In [ ]:
temp_dir_obj = tempfile.TemporaryDirectory()
temp_dir = temp_dir_obj.name
package_location = Ex.staging.prep_holmes_package(
    staging_path=temp_dir, 
    commit_hash='ca6ccd63f270d603ffa9bdf5a7ced184d6339796'
)

detects_dict={}
for video_data in Ex.val_data:
    detects_dict[video_data['video_id']] = video_data['video_path'].replace('processed','detections')

def_conf_path = Ex.staging.extract_default_config(package_location, temp_dir)

with open(def_conf_path, 'r') as fp:
    def_conf_dict = json.load(fp)

new_def_conf_dicts={}
list_dicts=[]
for video_data in Ex.val_data:
    new_def_conf_dict = def_conf_dict.copy()
    detects_path = video_data['video_path'].replace('processed','detections').replace('.mp4', '.json')
    new_def_conf_dict['detector_config'] = {"source_path": detects_path,"_class_name": "PassThroughDetectorConfig","batch_size": 1}
    new_def_conf_dict['video_config'] = video_data
    new_def_conf_dict['video_config']['_class_name'] = 'RawVideoConfig'
    list_dicts.append(new_def_conf_dict)

In [10]:
measurement_ids = ['M01-PS0304-001', 'M02-PP0214-000', 'M00-ChvUtd-000', 'M03-ER0429-001']
minimum_detections = [10,15,20,25,30,35,40,45,50]

In [ ]:
list_configs=[]
for min_det in minimum_detections:
    for config in list_dicts[3]:
        current_config_dict = config
        flat_dict = flatten_dict(current_config_dict)
        updated_config_dict = expand_dict(flat_dict)
        updated_config_dict["filter_configs"] = [{"_class_name": 'BbAreaFilterConfig', 'min_detections': min_det, "max_pixel_area": 5000, "min_pixel_area": 100}]
        file_name = os.path.join(temp_dir, "M{:02d}-{}-{:02d}.json".format(len(list_configs), 
                                            current_config_dict['video_config']['video_id'], min_det))
        list_configs.append(file_name)
        with open(file_name, 'w') as f:
            json.dump(updated_config_dict, f, indent=2)
print(len(list_configs), list_configs[0])

In [ ]:
experiment = Ex.AWSExperiment(
    experiment_id="PTA-95-DET-1-2", job_queue=Ex.AWSComputeQueue.cpu_queue
)

for i, config_path in enumerate(list_configs):
    meas_id = os.path.basename(config_path).replace(".json", "")
    experiment.add_measurement(
        Ex.AWSMeasurement(
            experiment_id = "PTA-95-DET-1-2",
            measurement_id = meas_id,
            job_queue = Ex.AWSComputeQueue.cpu_queue,
            package_path = package_location,
            config_path = config_path
        )
    )

In [13]:
import boto3
experiment._batch_client = boto3.client('batch', region_name='us-east-1')

In [ ]:
experiment.stage_experiment()

In [3]:
experiment.launch_experiment()

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/Users/jason.katz/Downloads/DET2.csv')

In [ ]:
from bokeh.charts import Scatter
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row

In [5]:
output_notebook()

Loading BokehJS ...

In [ ]:
data['min_det'] = 1
for index, row in data.iterrows():
    data['min_det'][index] = row['job_id'][-3:]
data['min_det'][data['min_det']<0]=data['min_det'][data['min_det']<0]*-1

In [ ]:
data['game'] = 'id'
for index, row in data.iterrows():
    data['game'][index] = row['job_id'][19:25]

In [8]:
scat1 = Scatter(data, x='min_det', y='mota', color='game')
show(scat1)

In [9]:
data_min_det = data.copy()
data_min_det['minimum_detections'] = data_min_det['min_det']
data_min_det = data_min_det.groupby(['min_det']).mean()
data_min_det

,mota,motp,motr,mostly_tracked,mt_auc,minimum_detections
min_det,,,,,,
10,0.648541,0.619226,0.825793,0.657555,0.802217,10
15,0.650479,0.619464,0.823941,0.656036,0.800839,15
20,0.651643,0.619612,0.821925,0.645406,0.799058,20
25,0.652650,0.620103,0.819445,0.642369,0.797019,25
30,0.653440,0.620322,0.816398,0.636295,0.794303,30
35,0.653390,0.620619,0.812767,0.634776,0.791031,35
40,0.653258,0.621119,0.808789,0.621868,0.787350,40
45,0.653347,0.621476,0.805116,0.608960,0.784015,45
50,0.651977,0.621846,0.800581,0.605163,0.779830,50


In [10]:
scat2 = Scatter(data_min_det, x = 'minimum_detections', y='mota')
scat3 = Scatter(data_min_det, x = 'minimum_detections', y='motp')
scat4 = Scatter(data_min_det, x = 'minimum_detections', y='mostly_tracked')
scat5 = Scatter(data_min_det, x = 'minimum_detections', y='mt_auc')
from bokeh.layouts import column, row
show(column(row(scat2, scat3), row(scat4, scat5)))